In [12]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

from functools import partial


folder = "physionet.org/files/"

In [13]:
def basic_parse_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    # Drop rows with missing values
    df = df.dropna()
    # Drop duplicate rows
    df = df.drop_duplicates(subset=["hadm_id", "icd_code"])
    return df


def parse_codes(df: pd.DataFrame) -> pd.DataFrame:
    # Split the icd_code into the first 3 characters and the rest
    df = df.dropna(subset=["icd_code"])
    df = df.drop_duplicates(subset=["hadm_id", "icd_code"])
    # Group by subject-id, hadm-id, and icd-version and aggregate the icd-codes into a list
    df = (
        df.groupby(["subject_id", "hadm_id", "icd_version"])
        .agg({"icd_code": list})
        .reset_index()
    )
    return df


def add_prefix(df: pd.DataFrame, prefix: str) -> pd.DataFrame:
    """For each code in the icd_column, add a string prefix to the front followed by a hyphen.
    
    Codes are assumed to be strings.
    Returns the resulting dataframe."""
    df["icd_code"] = df["icd_code"].apply(lambda x: prefix + "-" + x)
    return df

In [14]:
diagnoses_icd = pd.read_csv(
    folder + "mimiciv/2.2/hosp/diagnoses_icd.csv.gz",
    compression="gzip",
)

procedures_icd = pd.read_csv(
    folder + "mimiciv/2.2/hosp/procedures_icd.csv.gz",
    compression="gzip",
)

diagnoses_icd = basic_parse_dataframe(diagnoses_icd)
procedures_icd = basic_parse_dataframe(procedures_icd)

# Add the prefix to the icd_code
diagnoses_icd = add_prefix(diagnoses_icd, "d")
procedures_icd = add_prefix(procedures_icd, "p")

diagnoses_icd = parse_codes(diagnoses_icd)
procedures_icd = parse_codes(procedures_icd)

# Rename the icd_code column to icd_diag and icd_proc
diagnoses_icd = diagnoses_icd.rename(columns={"icd_code": "icd_diag"})
procedures_icd = procedures_icd.rename(columns={"icd_code": "icd_proc"})

In [15]:
# Split into ICD version 9 and version 10
diagnoses_icd9 = diagnoses_icd[diagnoses_icd.icd_version == 9]
diagnoses_icd10 = diagnoses_icd[diagnoses_icd.icd_version == 10]

procedures_icd9 = procedures_icd[procedures_icd.icd_version == 9]
procedures_icd10 = procedures_icd[procedures_icd.icd_version == 10]

In [16]:
diagnoses_icd9.head()

,subject_id,hadm_id,icd_version,icd_diag
0,10000032,22595853,9,"[d-5723, d-78959, d-5715, d-07070, d-496, d-29..."
1,10000032,22841357,9,"[d-07071, d-78959, d-2875, d-2761, d-496, d-57..."
2,10000032,25742920,9,"[d-07054, d-78959, d-V462, d-5715, d-2767, d-2..."
3,10000032,29079034,9,"[d-45829, d-07044, d-7994, d-2761, d-78959, d-..."
4,10000068,25022803,9,[d-30500]


In [17]:
procedures_icd10.head()

,subject_id,hadm_id,icd_version,icd_proc
4,10000117,27988844,10,[p-0QS734Z]
28,10001401,21544441,10,"[p-0TTB4ZZ, p-07BC4ZX, p-0UT9FZZ, p-0UTC7ZZ, p..."
29,10001401,26840593,10,"[p-02HV33Z, p-3E0436Z, p-0W9J30Z]"
30,10001401,27012892,10,"[p-0T948ZX, p-0TB68ZX, p-0T783DZ, p-02HV33Z]"
31,10001401,27060146,10,"[p-0D9W3ZX, p-0DWW30Z, p-0D9W30Z, p-02HV33Z]"


In [18]:
# Get total label count. This is the number of diagnoses and procedure codes, seperated into icd9 and icd10

diagnoses_codes = pd.read_csv(
    folder + "mimiciv/2.2/hosp/d_icd_diagnoses.csv.gz",
    compression="gzip",
)

procedure_codes = pd.read_csv(
    folder + "mimiciv/2.2/hosp/d_icd_procedures.csv.gz",
    compression="gzip",
)

# Add the prefix to the icd_code
diagnoses_codes = add_prefix(diagnoses_codes, "d")
procedure_codes = add_prefix(procedure_codes, "p")

# Split into ICD version 9 and version 10
icd9_diagnoses_codes = diagnoses_codes[diagnoses_codes.icd_version == 9]
icd10_diagnoses_codes = diagnoses_codes[diagnoses_codes.icd_version == 10]

icd9_procedures_codes = procedure_codes[procedure_codes.icd_version == 9]
icd10_procedures_codes = procedure_codes[procedure_codes.icd_version == 10]

# Merge the diagnoses and procedures
icd9_codes = pd.concat([icd9_diagnoses_codes, icd9_procedures_codes])
icd10_codes = pd.concat([icd10_diagnoses_codes, icd10_procedures_codes])

# Print counts entries
print("ICD9 count:", len(icd9_codes))
print("ICD10 count:", len(icd10_codes))

ICD9 count: 18554
ICD10 count: 176478


In [19]:
# Print rows where hadm_id is "25022803"
diagnoses_icd9[diagnoses_icd9.hadm_id == 25022803]

,subject_id,hadm_id,icd_version,icd_diag
4,10000068,25022803,9,[d-30500]


In [20]:
icd9_codes.head()

,icd_code,icd_version,long_title
0,d-0010,9,Cholera due to vibrio cholerae
1,d-0011,9,Cholera due to vibrio cholerae el tor
2,d-0019,9,"Cholera, unspecified"
3,d-0020,9,Typhoid fever
4,d-0021,9,Paratyphoid fever A


In [21]:
mimic_notes = pd.read_csv(
    folder + "mimic-iv-note/2.2/note/discharge.csv.gz",
    compression="gzip",
    usecols=lambda x: x not in ["storetime", "note_type", "charttime"],
)
# Drop missing rows that are missing "text" or "hadm_id" or "subject_id"
mimic_notes = mimic_notes.dropna(subset=["text", "hadm_id", "subject_id"])

mimic_notes.head()

,note_id,subject_id,hadm_id,note_seq,text
0,10000032-DS-21,10000032,22595853,21,\nName: ___ Unit No: _...
1,10000032-DS-22,10000032,22841357,22,\nName: ___ Unit No: _...
2,10000032-DS-23,10000032,29079034,23,\nName: ___ Unit No: _...
3,10000032-DS-24,10000032,25742920,24,\nName: ___ Unit No: _...
4,10000084-DS-17,10000084,23052089,17,\nName: ___ Unit No: __...


In [22]:
mimic_9 = mimic_notes.merge(
    diagnoses_icd9[["hadm_id", "icd_diag"]], on="hadm_id", how="left"
)

mimic_9 = mimic_9.merge(
    procedures_icd9[["hadm_id", "icd_proc"]], on="hadm_id", how="left"
)

mimic_10 = mimic_notes.merge(
    diagnoses_icd10[["hadm_id", "icd_diag"]], on="hadm_id", how="left"
)

mimic_10 = mimic_10.merge(
    procedures_icd10[["hadm_id", "icd_proc"]], on="hadm_id", how="left"
)

# Remove notes with no icd code
mimic_9 = mimic_9.dropna(subset=["icd_diag", "icd_proc"])
mimic_10 = mimic_10.dropna(subset=["icd_diag", "icd_proc"])

mimic_9.head()

,note_id,subject_id,hadm_id,note_seq,text,icd_diag,icd_proc
0,10000032-DS-21,10000032,22595853,21,\nName: ___ Unit No: _...,"[d-5723, d-78959, d-5715, d-07070, d-496, d-29...",[p-5491]
1,10000032-DS-22,10000032,22841357,22,\nName: ___ Unit No: _...,"[d-07071, d-78959, d-2875, d-2761, d-496, d-57...",[p-5491]
3,10000032-DS-24,10000032,25742920,24,\nName: ___ Unit No: _...,"[d-07054, d-78959, d-V462, d-5715, d-2767, d-2...",[p-5491]
8,10000560-DS-15,10000560,28979390,15,\nName: ___ Unit No: _...,"[d-1890, d-V1582, d-V1201]",[p-5551]
10,10000826-DS-17,10000826,20032235,17,\nName: ___. Unit No: ___\n \nAdm...,"[d-5712, d-486, d-78959, d-5723, d-5990, d-263...",[p-5491]


In [23]:
# Create label column
mimic_9["label"] = mimic_9["icd_diag"] + mimic_9["icd_proc"]
mimic_10["label"] = mimic_10["icd_diag"] + mimic_10["icd_proc"]

mimic_9.head()

,note_id,subject_id,hadm_id,note_seq,text,icd_diag,icd_proc,label
0,10000032-DS-21,10000032,22595853,21,\nName: ___ Unit No: _...,"[d-5723, d-78959, d-5715, d-07070, d-496, d-29...",[p-5491],"[d-5723, d-78959, d-5715, d-07070, d-496, d-29..."
1,10000032-DS-22,10000032,22841357,22,\nName: ___ Unit No: _...,"[d-07071, d-78959, d-2875, d-2761, d-496, d-57...",[p-5491],"[d-07071, d-78959, d-2875, d-2761, d-496, d-57..."
3,10000032-DS-24,10000032,25742920,24,\nName: ___ Unit No: _...,"[d-07054, d-78959, d-V462, d-5715, d-2767, d-2...",[p-5491],"[d-07054, d-78959, d-V462, d-5715, d-2767, d-2..."
8,10000560-DS-15,10000560,28979390,15,\nName: ___ Unit No: _...,"[d-1890, d-V1582, d-V1201]",[p-5551],"[d-1890, d-V1582, d-V1201, p-5551]"
10,10000826-DS-17,10000826,20032235,17,\nName: ___. Unit No: ___\n \nAdm...,"[d-5712, d-486, d-78959, d-5723, d-5990, d-263...",[p-5491],"[d-5712, d-486, d-78959, d-5723, d-5990, d-263..."


In [25]:
# Check for codes shared between icd9 diagnoses and procedures
shared_codes_9 = set(icd9_diagnoses_codes.icd_code).intersection(
    set(icd9_procedures_codes.icd_code)
)
print("Shared codes:", len(shared_codes_9))

# Check for codes shared between icd10 diagnoses and procedures
shared_codes_10 = set(icd10_diagnoses_codes.icd_code).intersection(
    set(icd10_procedures_codes.icd_code)
)

print("Shared codes:", len(shared_codes_10))

Shared codes: 0
Shared codes: 0


In [26]:
# Report the number of entries, the average length of labels, average length of text, and the number of labels that occur more than 5 times

print("MIMIC-9")
print("Number of entries:", len(mimic_9))
print("Average length of labels:", np.mean(mimic_9.label.apply(len)))
print("Average length of text:", np.mean(mimic_9.text.apply(len)))

print("MIMIC-10")
print("Number of entries:", len(mimic_10))
print("Average length of labels:", np.mean(mimic_10.label.apply(len)))
print("Average length of text:", np.mean(mimic_10.text.apply(len)))

MIMIC-9
Number of entries: 126863
Average length of labels: 15.03067876370573
Average length of text: 10017.60032475978
MIMIC-10
Number of entries: 67934
Average length of labels: 18.576162157388055
Average length of text: 12071.17162245709
